# Load and cleanup data


We load user activity data and cloud cost data for the duration of the semester
into pandas dataframes. We make sure our timezones are right, and that we fill in
NaNs where we have missing data.

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# Log data for user activity
path_usage = '../data/processed/fall-2018/user-activity.jsonl'
datahub = pd.read_json(path_usage, lines=True)

# Index by timestamp
datahub = datahub.set_index('timestamp')

# Our timestamps are in UTC
datahub = datahub.tz_localize('UTC')
# But we want them in US/Pacific, since that's how our billing data is setup
datahub = datahub.tz_convert('US/Pacific')

In [3]:
# Turn actions into sessions (takes a moment)
# This makes the assumption that a session always begins with a "start" action
# and ends with the next "stop" action by the same user.
# It throws away sessions that begin with a "stop" action, or do not end w/ a "stop" action

sessions = []
active_sessions = {}

rows = list(datahub.reset_index().iterrows())
for ii, irow in tqdm(rows):
    if irow['user'] not in active_sessions:
        if irow['action'] == 'start':
            active_sessions[irow['user']] = [irow['timestamp']]
    else:
        active_sessions[irow['user']].append(irow['timestamp'])
        sessions.append(active_sessions.pop(irow['user']) + [irow['user']])
sessions = pd.DataFrame(sessions, columns=['start', 'stop', 'user'])

100%|██████████| 462456/462456 [00:24<00:00, 18865.50it/s]


In [4]:
# Length is in minutes
sessions['length'] = (sessions['stop'] - sessions['start'])

# Smallest unit of time is an hour, so convert length to hours
sessions['length_hours'] = sessions['length'].map(lambda a: a.seconds / 60 / 60)

# Drop the timedelta since it's messy and redundant
sessions = sessions.drop(columns=['length'])

In [5]:
# Save to json
sessions.to_json('../data/features/fall-2018/user-sessions.jsonl')